In [1]:
from utils import evaluate_policy, str2bool, Actor, Double_Q_Critic, ReplayBuffer, test_policy, Reward_adapter
from datetime import datetime
import gymnasium as gym
import os, shutil
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from torch.utils.tensorboard import SummaryWriter
import random
import numpy as np
import copy
import math
from collections import deque

from torch.distributions import Beta,Normal

In [2]:
def get_args():
    # 创建命令行参数解析器
    parser = argparse.ArgumentParser()

    # 添加各种命令行参数
    parser.add_argument('--algo_name',default='TD3',type=str,help="算法名")
    parser.add_argument('--dvc', type=str, default='cuda', help='运行设备: cuda 或 cpu')
    parser.add_argument('--env_name', type=str, default='HumanoidStandup-v4', help='环境名')
    parser.add_argument('--render_mode', type=str, default='rgb_array', help='环境渲染模式')
    parser.add_argument('--write', type=str2bool, default=True, help='使用SummaryWriter记录训练')
    parser.add_argument('--render', type=str2bool, default=False, help='是否渲染')
    parser.add_argument('--Loadmodel', type=str2bool, default=False, help='是否加载预训练模型')
    parser.add_argument('--ModelIdex', type=int, default=94000, help='要加载的模型索引')
    parser.add_argument('--deque_maxlen',default=20,type=int)

    parser.add_argument('--seed', type=int, default=1, help='随机种子')
    parser.add_argument('--Max_train_steps', type=int, default=5e7, help='最大训练步数')
    parser.add_argument('--save_interval', type=int, default=5e4, help='模型保存间隔，以步为单位')
    parser.add_argument('--eval_interval', type=int, default=2e3, help='模型评估间隔，以步为单位')
    parser.add_argument('--test_interval', type=int, default=5e4, help='视频保存间隔，以步为单位')
    parser.add_argument('--update_every', type=int, default=50, help='training frequency')

    parser.add_argument('--gamma', type=float, default=0.99, help='折扣因子')
    parser.add_argument('--net_width', type=int, default=256, help='隐藏网络宽度')
    parser.add_argument('--a_lr', type=float, default=1e-4, help='Learning rate of actor')
    parser.add_argument('--c_lr', type=float, default=1e-4   , help='Learning rate of critic')
    parser.add_argument('--batch_size', type=int, default=256, help='切片轨迹的长度')
    parser.add_argument('--random_steps', type=int, default=500, help='切片轨迹的长度')
    parser.add_argument('--explore_noise', type=float, default=0.15, help='exploring noise when interacting')
    parser.add_argument('--explore_noise_decay', type=float, default=0.998, help='Decay rate of explore noise')
    parser.add_argument('--delay_freq', type=int, default=1, help='Delayed frequency for Actor and Target Net')
    
    # 解析命令行参数
    args = parser.parse_args([])
    args = {**vars(args)}  # 转换成字典类型    
    
    return args

In [3]:
def print_args(args):
    ## 打印超参数
    print("超参数")
    print(''.join(['=']*80))
    tplt = "{:^20}\t{:^20}\t{:^20}"
    print(tplt.format("Name", "Value", "Type"))
    for k,v in args.items():
        print(tplt.format(k,v,str(type(v))))   
    print(''.join(['=']*80))  

In [4]:
def all_seed(env, seed):
    """
    设置随机种子以确保实验的可重复性

    参数:
    - env: Gym 环境，用于训练模型
    - seed: 随机种子值

    说明:
    1. 使用给定的随机种子设置 NumPy、Python、PyTorch 和 CUDA 的随机生成器。
    2. 禁用 CUDA 的非确定性操作以确保实验结果的一致性。
    """

    np.random.seed(seed)  # 设置 NumPy 随机种子
    random.seed(seed)  # 设置 Python 随机种子
    torch.manual_seed(seed)  # 设置 PyTorch 随机种子
    torch.cuda.manual_seed(seed)  # 设置 PyTorch CUDA 随机种子
    os.environ['PYTHONHASHSEED'] = str(seed)  # 设置 Python Hash 随机种子
    torch.backends.cudnn.deterministic = True  # 禁用 CUDA 非确定性操作以确保实验结果的一致性
    torch.backends.cudnn.benchmark = False  # 禁用 CUDA 非确定性操作以确保实验结果的一致性
    torch.backends.cudnn.enabled = False  # 禁用 CUDA 非确定性操作以确保实验结果的一致性

In [5]:
class TD3_agent():
    def __init__(self, kwargs):
        """
        初始化强化学习代理

        参数:
        - kwargs: 包含初始化参数的字典

        说明:
        1. 更新对象的属性，使用传递的关键字参数。
        2. 设置策略噪声、噪声裁剪和软更新参数。
        3. 初始化演员网络，优化器，以及演员网络的目标网络。
        4. 初始化双 Q 评论者网络，优化器，以及双 Q 评论者网络的目标网络。
        5. 初始化经验回放缓冲区。
        """

        self.__dict__.update(kwargs)  # 1. 更新对象的属性，使用传递的关键字参数。
        self.policy_noise = 0.2 * self.max_action  # 2. 设置策略噪声
        self.noise_clip = 0.5 * self.max_action  # 设置噪声裁剪
        self.tau = 0.005  # 设置软更新参数
        self.delay_counter = 0

        self.actor = Actor(self.state_dim, self.action_dim, self.net_width, self.max_action).to(self.dvc)  # 3. 初始化演员网络
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=self.a_lr)  # 初始化演员网络的优化器
        self.actor_target = copy.deepcopy(self.actor)  # 初始化演员网络的目标网络

        self.q_critic = Double_Q_Critic(self.state_dim, self.action_dim, self.net_width).to(self.dvc)  # 4. 初始化双 Q 评论者网络
        self.q_critic_optimizer = torch.optim.Adam(self.q_critic.parameters(), lr=self.c_lr)  # 初始化双 Q 评论者网络的优化器
        self.q_critic_target = copy.deepcopy(self.q_critic)  # 初始化双 Q 评论者网络的目标网络

        self.replay_buffer = ReplayBuffer(self.state_dim, self.action_dim, max_size=int(1e6), dvc=self.dvc)  # 5. 初始化经验回放缓冲区

    def select_action(self, state, deterministic):
        """
        选择动作

        参数:
        - state: 当前状态
        - deterministic: 是否使用确定性策略选择动作

        返回:
        - 动作

        说明:
        1. 将状态从 [x, x, ..., x] 转换为 [[x, x, ..., x]]。
        2. 通过 Actor 网络获取动作。
        3. 如果是确定性策略，则直接返回动作。
        4. 如果是非确定性策略，则添加噪声并返回，确保在动作范围内。
        """
        with torch.no_grad():
            state = torch.FloatTensor(state[np.newaxis, :]).to(self.dvc)  # 1. 将状态转换为网络输入格式
            a = self.actor(state).cpu().numpy()[0]  # 2. 获取动作

            if deterministic:
                return a  # 3. 返回动作（确定性策略）

            else:
                noise = np.random.normal(0, self.max_action * self.explore_noise, size=self.action_dim)  # 4. 生成噪声
                return (a + noise).clip(-self.max_action, self.max_action)  # 5. 添加噪声并确保在动作范围内
        
    def train(self):
        """
        训练双 Q 评论者和演员网络

        说明:
        1. 更新延迟计数器。
        2. 从经验回放缓冲区中采样状态、动作、奖励、下一个状态和是否结束的标志。
        3. 生成目标动作的噪声并应用裁剪。
        4. 计算平滑目标动作和目标 Q 值。
        5. 计算目标 Q 值。
        6. 计算当前 Q 值。
        7. 计算 Q 评论者网络的损失。
        8. 执行 Q 评论者网络的优化步骤。
        9. 如果延迟计数器大于延迟频率，更新演员网络。
        10. 执行演员网络的优化步骤。
        11. 执行软更新，更新目标网络参数。
        12. 重置延迟计数器。
        """

        # 1. 更新延迟计数器。
        self.delay_counter += 1
        
        a_loss = 0
        q_loss = 0

        # 2. 从经验回放缓冲区中采样状态、动作、奖励、下一个状态和是否结束的标志。
        with torch.no_grad():
            s, a, r, s_next, dw = self.replay_buffer.sample(self.batch_size)

            # 3. 生成目标动作的噪声并应用裁剪。
            target_a_noise = (torch.randn_like(a) * self.policy_noise).clamp(-self.noise_clip, self.noise_clip)

            # 4. 计算平滑目标动作。
            smoothed_target_a = (self.actor_target(s_next) + target_a_noise).clamp(-self.max_action, self.max_action)
            # 让两个目标价值网络做预测。
            target_Q1, target_Q2 = self.q_critic_target(s_next, smoothed_target_a)

            # 5. 计算 TD 目标。
            target_Q = torch.min(target_Q1, target_Q2)
            target_Q = r + (~dw) * self.gamma * target_Q  # dw: die or win

        # 6. 让两个价值网络做预测。
        current_Q1, current_Q2 = self.q_critic(s, a)

        # 7. 计算 Q 评论者网络的损失。
        q_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)
        self.q_critic_optimizer.zero_grad()
        q_loss.backward()
        self.q_critic_optimizer.step()
        q_loss = q_loss.item()

        if self.delay_counter > self.delay_freq:
            # 9. 如果延迟计数器大于延迟频率，更新演员网络。
            # Update the Actor
            a_loss = -self.q_critic.Q1(s, self.actor(s)).mean()
            self.actor_optimizer.zero_grad()
            a_loss.backward()
            self.actor_optimizer.step()
            a_loss = a_loss.item()

            # 11. 执行软更新，更新目标网络参数。
            with torch.no_grad():
                for param, target_param in zip(self.q_critic.parameters(), self.q_critic_target.parameters()):
                    target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

                for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                    target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

            # 12. 重置延迟计数器。
            self.delay_counter = 0
            
        return a_loss, q_loss

    def save(self, episode):
        """
        保存当前训练模型的Actor和Critic参数到文件

        参数:
        - episode: 当前训练的episode数，用于在文件名中标识不同的保存点
        """
        model_path = f"model/{cfg['path']}"
        # 检查是否存在'model'文件夹，如果不存在则创建
        try:
            os.makedirs(model_path)
        except FileExistsError:
            pass
        # 保存Critic的参数到文件
        torch.save(self.q_critic.state_dict(), f"{model_path}/ddpg_critic{episode}.pth")
        # 保存Actor的参数到文件
        torch.save(self.actor.state_dict(), f"{model_path}/ppo_actor{episode}.pth")

    def load(self, episode):
        """
        从文件加载之前保存的Actor和Critic参数

        参数:
        - episode: 要加载的保存点的episode数
        """
        model_path = f"model/{cfg['path']}"
        # 加载之前保存的Critic的参数
        self.q_critic.load_state_dict(torch.load(f"{model_path}/ddpg_critic{episode}.pth"))
        # 加载之前保存的Actor的参数
        self.actor.load_state_dict(torch.load(f"{model_path}/ppo_actor{episode}.pth"))

In [6]:
def env_agent_config(cfg, path):
    """
    配置环境和代理

    参数:
    - cfg: 包含配置信息的字典
    - path: 模型保存路径

    返回:
    - env: Gym 环境
    - agent: PPO 代理

    说明:
    1. 创建指定环境并设置渲染模式。
    2. 如果配置中设置了种子，则为环境设置种子。
    3. 获取环境的状态空间维度和动作空间维度。
    4. 更新配置字典中的状态维度和动作维度。
    5. 创建 PPO 代理。

    注意:
    - PPO 代理的创建依赖于配置信息和模型保存路径。
    """
    env = gym.make(cfg['env_name'], render_mode=cfg['render_mode'])  # 1. 创建环境
    eval_env = gym.make(cfg['env_name'], render_mode=cfg['render_mode'])
    if cfg['seed'] != 0:
        all_seed(env, seed=cfg['seed'])  # 2. 如果配置中设置了种子，则为环境设置种子

    n_states = env.observation_space.shape[0]  # 3. 获取状态空间维度
    n_actions = env.action_space.shape[0]  # 获取动作空间维度
    max_action = float(env.action_space.high[0]) # 获取动作空间的最大值
    max_e_steps = env._max_episode_steps  # 最大步数
    print(f"状态空间维度：{n_states}，动作空间维度：{n_actions}，最大步数：{max_e_steps}")
    cfg.update({"state_dim": n_states, "action_dim": n_actions, "max_e_steps": max_e_steps, "max_action": max_action})  # 4. 更新n_states和n_actions到cfg参数中

    agent = TD3_agent(cfg)  # 5. 创建 PPO 代理
    return env, eval_env, agent

In [7]:
cfg = get_args()

path = f"device:{cfg['dvc']}/{cfg['env_name']}/seed:{cfg['seed']}/{cfg['algo_name']}/net_width-{cfg['net_width']}-gamma-{cfg['gamma']}-a_lr-{cfg['a_lr']}-c_lr-{cfg['c_lr']}-batch_size-{cfg['batch_size']}-delay_freq-{cfg['delay_freq']}"
cfg.update({"path":path}) # 更新n_states和n_actions到cfg参数中

base_dir = f"log/{cfg['path']}"

env, eval_env, agent = env_agent_config(cfg, path)

cfg.update({"mean_break":4e1000})

print_args(cfg)

状态空间维度：376，动作空间维度：17，最大步数：1000
超参数
        Name        	       Value        	        Type        
     algo_name      	        TD3         	   <class 'str'>    
        dvc         	        cuda        	   <class 'str'>    
      env_name      	 HumanoidStandup-v4 	   <class 'str'>    
    render_mode     	     rgb_array      	   <class 'str'>    
       write        	         1          	   <class 'bool'>   
       render       	         0          	   <class 'bool'>   
     Loadmodel      	         0          	   <class 'bool'>   
     ModelIdex      	       94000        	   <class 'int'>    
    deque_maxlen    	         20         	   <class 'int'>    
        seed        	         1          	   <class 'int'>    
  Max_train_steps   	     50000000.0     	  <class 'float'>   
   save_interval    	      50000.0       	  <class 'float'>   
   eval_interval    	       2000.0       	  <class 'float'>   
   test_interval    	      50000.0       	  <class 'float'>   
    update_every    

In [8]:
def train(cfg):
    print("开始训练")
    env_seed = cfg['seed']
    # 使用TensorBoard记录训练曲线
    if cfg['write']:
        writepath = 'runs/{}'.format(cfg['path']) # 构建TensorBoard日志路径
        if os.path.exists(writepath): shutil.rmtree(writepath)  # 如果路径已存在，则删除该路径及其内容
        writer = SummaryWriter(log_dir=writepath)  # 创建TensorBoard写入器，指定日志路径

    # 如果指定了加载模型的选项，则加载模型
    if cfg['Loadmodel']:
        print("加载模型")
        agent.load(cfg['ModelIdex'])

    # 如果选择渲染模式
    if cfg['render']:
        while True:
            # 在环境中评估智能体的性能，并输出奖励
            ep_r = evaluate_policy(env, agent, turns=1)
            print('Env: ', cfg['env_name'],' Episode Reward: ', {ep_r})
    else:
        total_steps = 0
        scores_deque = deque(maxlen=cfg['deque_maxlen'])
        a_loss_deque = deque(maxlen=cfg['update_every'])
        c_loss_deque = deque(maxlen=cfg['update_every'])

        # 在达到最大训练步数前一直进行训练
        while total_steps < cfg['Max_train_steps']:
            # 重置环境，获取初始状态
            s, info = env.reset(seed=env_seed)  # 重置环境，使用环境种子
            env_seed += 1
            done = False

            # 与环境进行交互并训练
            while not done:
                if total_steps < cfg['random_steps']:
                    a = env.action_space.sample()
                else:
                    a = agent.select_action(s, deterministic=False)
                # 选择动作和动作对应的对数概率
                s_next, r, dw, tr, info = env.step(a) # 与环境交互
                r = Reward_adapter(r, cfg['env_name'], s, s_next)  # 调整奖励
                done = (dw or tr)  # 如果游戏结束（死亡或胜利），则done为True

                # 存储当前的转移数据
                agent.replay_buffer.add(s, a, r, s_next, dw)
                s = s_next
                total_steps += 1

                # 如果达到更新时间
                # if total_steps >= cfg['random_steps']:
                if (total_steps >= cfg['random_steps']) and (total_steps % cfg['update_every'] == 0):
                    for j in range(cfg['update_every']):
                        a_loss, c_loss = agent.train()  # 执行PPO算法的训练步骤
                        a_loss_deque.append(a_loss)
                        c_loss_deque.append(c_loss)
                    if cfg['write']:
                        writer.add_scalar('Loss_a', np.mean(a_loss_deque), global_step=total_steps)
                        writer.add_scalar('Loss_c', np.mean(c_loss_deque), global_step=total_steps)

                # 如果达到记录和日志的时间
                if (total_steps % cfg['eval_interval'] == 0) and (total_steps >= cfg['random_steps']):
                    agent.explore_noise *= cfg['explore_noise_decay'] # 衰减策略噪声
                    # 在评估环境中评估智能体，并输出平均奖励
                    score = evaluate_policy(eval_env, agent, turns=3, cfg=cfg)  # 对策略进行3次评估，取平均值
                    scores_deque.append(score)
                    if cfg['write']:
                        writer.add_scalar('Score_ep', score, global_step=total_steps)  # 将评估得分记录到TensorBoard
                        writer.add_scalar('Score_Average', np.mean(scores_deque), global_step=total_steps)
                    print('EnvName:', cfg['env_name'], 'seed:', cfg['seed'],
                          'steps: {}k'.format(int(total_steps / 1000)), 'score:', score)
                    
                if (total_steps % cfg['test_interval'] == 0) and (total_steps >= cfg['random_steps']):
                    print("测试模型")
                    test_policy(eval_env, agent, total_steps, turns=1, path=cfg['path'], cfg=cfg)

                # 如果达到保存模型的时间
                if (total_steps % cfg['save_interval']  == 0) and (total_steps >= cfg['random_steps']):
                    print("保存模型")
                    agent.save(total_steps)  # 保存模型

                if (np.mean(scores_deque) >= cfg['mean_break']) and (len(scores_deque) >= cfg['deque_maxlen']):
                    print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(total_steps, np.mean(scores_deque)))
                    test_policy(eval_env, agent, total_steps, turns=1, path=cfg['path'], cfg=cfg)
                    print("保存模型")
                    agent.save(total_steps)
                    env.close()
                    eval_env.close()
                    return

        env.close()
        eval_env.close()

In [9]:
train(cfg)

开始训练


/home/q1001p/anaconda3/envs/DRL_3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/q1001p/anaconda3/envs/DRL_3.11/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


EnvName: HumanoidStandup-v4 seed: 1 steps: 2k score: 31389
EnvName: HumanoidStandup-v4 seed: 1 steps: 4k score: 38875
EnvName: HumanoidStandup-v4 seed: 1 steps: 6k score: 36091
EnvName: HumanoidStandup-v4 seed: 1 steps: 8k score: 35685
EnvName: HumanoidStandup-v4 seed: 1 steps: 10k score: 39436
EnvName: HumanoidStandup-v4 seed: 1 steps: 12k score: 34080
EnvName: HumanoidStandup-v4 seed: 1 steps: 14k score: 32988
EnvName: HumanoidStandup-v4 seed: 1 steps: 16k score: 34696
EnvName: HumanoidStandup-v4 seed: 1 steps: 18k score: 49671
EnvName: HumanoidStandup-v4 seed: 1 steps: 20k score: 30775
EnvName: HumanoidStandup-v4 seed: 1 steps: 22k score: 32980
EnvName: HumanoidStandup-v4 seed: 1 steps: 24k score: 32795
EnvName: HumanoidStandup-v4 seed: 1 steps: 26k score: 62166
EnvName: HumanoidStandup-v4 seed: 1 steps: 28k score: 33642
EnvName: HumanoidStandup-v4 seed: 1 steps: 30k score: 37282
EnvName: HumanoidStandup-v4 seed: 1 steps: 32k score: 42721
EnvName: HumanoidStandup-v4 seed: 1 steps: 3

Traceback (most recent call last):
  File "/home/q1001p/anaconda3/envs/DRL_3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_18024/1065089074.py", line 1, in <module>
    train(cfg)
  File "/tmp/ipykernel_18024/1046902105.py", line 54, in train
    a_loss, c_loss = agent.train()  # 执行PPO算法的训练步骤
                     ^^^^^^^^^^^^^
  File "/tmp/ipykernel_18024/859177947.py", line 109, in train
    self.q_critic_optimizer.step()
  File "/home/q1001p/anaconda3/envs/DRL_3.11/lib/python3.11/site-packages/torch/optim/optimizer.py", line 373, in wrapper
    out = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "/home/q1001p/anaconda3/envs/DRL_3.11/lib/python3.11/site-packages/torch/optim/optimizer.py", line 76, in _use_grad
    ret = func(self, *args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/q1001p/anaconda3/envs/DRL_3.11/lib/python3.11/site-pa